In [ ]:
%pip install -q ultralytics opencv-python
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import os
import time

# 1. Start timer
start_time = time.time()

# 2. Load video
video_path = "../utilities/cars_on_the_highway.mp4"
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found. Please check the path.")

# 3. Load YOLO model for detection
model_path = "../utilities/yolov8n.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file {model_path} not found. Please check the path.")

model = YOLO(model_path)

cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

print(f"Detected video resolution: {frame_width}x{frame_height}")

# 4. Run tracking
# Note: The model will automatically detect objects and track them.
results = model.track(
    source=video_path,
    conf=0.4,  # TEST: lowered it from 0.4, to see if there would be more entries in the csv
    iou=0.5,
    show=False,           # If True: display during run
    save=True,            # Saved video to 'runs/track'
    save_txt=True,        # Saving detection data in ".txt" format
    classes=[2, 5, 7],     # car, bus, truck
    tracker="botsort.yaml"  # or "bytetrack.yaml", "botsort.yaml", etc.
)

# 5. Converting results from .txt to .csv format
# Files: runs/track/predict/labels/frame_x.txt

labels_dir = "runs/detect/track/labels"
tracked_data = []

for file in sorted(os.listdir(labels_dir)):
    if not file.endswith(".txt"):
        continue
    # e.g.: "cars_on_the_highway_7.txt" → frame_idx = 7
    frame_idx = int(file.rsplit("_", 1)[1].split(".")[0])
    with open(os.path.join(labels_dir, file), "r") as f:
        for line in f.readlines():
            obj = line.strip().split()
            if len(obj) != 6:
                print(f"Fájl kihagyva, hibás sor: {line}")
                continue
            class_id = int(obj[0])
            x_center = float(obj[1])
            y_center = float(obj[2])
            width = float(obj[3])
            height = float(obj[4])
            track_id = int(obj[5])

            # YOLO-format -> pixel coords (based on real frame dimensions)
            x1 = int((x_center - width / 2) * frame_width)
            y1 = int((y_center - height / 2) * frame_height)
            x2 = int((x_center + width / 2) * frame_width)
            y2 = int((y_center + height / 2) * frame_height)

            tracked_data.append([frame_idx, track_id, x1, y1, x2, y2])

# 6. End timer
end_time = time.time()
runtime = end_time - start_time
fps = total_frames / runtime if runtime > 0 else 0

# 7. Save performance log
with open("yolov8_performance.txt", "w") as f:
    f.write(f"{runtime:.2f}\n")
    f.write(f"{fps:.2f}")

# 8. Save tracking data to CSV
columns = ["frame", "id", "x1", "y1", "x2", "y2"]
df = pd.DataFrame(tracked_data, columns=columns)
df.to_csv("trajectory_data_yolov8.csv", index=False)
print("Tracking data saved to trajectory_data_yolov8.csv")


Detected video resolution: 1920x1080

WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1513) c:\Users\HP\Documents\geometric-data-analysis\3_yolov8\..\utilities\cars_on_the_highway.mp4: 384x640 1 car, 59.1ms
video 1/1 (frame 2/1513) c:\Users\HP\Documents\geometric-data-analysis\3_yolov8\..\utilities\cars_on_the_highway.mp4: 384x640 1 car, 57.1ms
video 1/1 (frame 3/1513) c:\Users\HP\Documents\geometric-data-analysis\3_yolov8\..\utilities\cars_on_the_highway.mp4: 384x640 1 car, 58.4ms
vi